In [1]:
import pandas as pd
import numpy as np

In [2]:
df_colorectal_161 = pd.read_csv('/home/alexandra/PyTrial/data/Colorectal_161/Colorectal_161.csv')
df_colorectal_201 = pd.read_csv('/home/alexandra/PyTrial/data/Colorectal_201/Colorectal_201.csv')
df_colorectal_309 = pd.read_csv('/home/alexandra/PyTrial/data/Colorectal_309/Colorectal_309.csv')
df_esophagael_213 = pd.read_csv('/home/alexandra/PyTrial/data/Esophagael_213/Esophagael_213.csv')
df_lung_218 = pd.read_csv('/home/alexandra/PyTrial/data/Lung_218/Lung_218.csv')
df_lung_261 = pd.read_csv('/home/alexandra/PyTrial/data/Lung_261/Lung_261.csv')

In [ ]:
def load_features_from_file(file_path):
    """Load feature names from a file into a list."""
    with open(file_path, 'r') as file:
        # Read lines, strip whitespace, and filter out any empty lines
        features = [line.strip() for line in file if line.strip()]
    return features

binary_features161 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_161/binary_feature.txt')
numerical_features161 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_161/numerical_feature.txt')

binary_features201 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_201/binary_feature.txt')
numerical_features201 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_201/numerical_feature.txt')

binary_features309 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_309/binary_feature.txt')
numerical_features309 = load_features_from_file('/home/alexandra/PyTrial/data/Colorectal_309/numerical_feature.txt')

binary_features213 = load_features_from_file('/home/alexandra/PyTrial/data/Esophagael_213/binary_feature.txt')
numerical_features213 = load_features_from_file('/home/alexandra/PyTrial/data/Esophagael_213/numerical_feature.txt')

binary_features218 = load_features_from_file('/home/alexandra/PyTrial/data/Lung_218/binary_feature.txt')
numerical_features218 = load_features_from_file('/home/alexandra/PyTrial/data/Lung_218/numerical_feature.txt')

binary_features261 = load_features_from_file('/home/alexandra/PyTrial/data/Lung_261/binary_feature.txt')
numerical_features261 = load_features_from_file('/home/alexandra/PyTrial/data/Lung_261/numerical_feature.txt')


In [3]:
def linearize(row, binary_features, numerical_features):
    result = []
    for col, val in row.items():
        if col == "id" or col == 'target_label':
            continue
        if col in binary_features:
            if val == "Yes":
                result.append(f"{col}: {val}")
        elif col in numerical_features:
            result.append(f"{col}: {val}")
        elif val =='indeterminate':
            continue
        else:
            result.append(f"{col}: {val}")
    return ", ".join(result)

In [ ]:
df_colorectal_161['linearized'] = df_colorectal_161.apply(lambda row: linearize(row, binary_features161, numerical_features161), axis=1)
df_colorectal_201['linearized'] = df_colorectal_201.apply(lambda row: linearize(row, binary_features201, numerical_features201), axis=1)
df_colorectal_309['linearized'] = df_colorectal_309.apply(lambda row: linearize(row, binary_features309, numerical_features309), axis=1)
df_esophagael_213['linearized'] = df_esophagael_213.apply(lambda row: linearize(row, binary_features213, numerical_features213), axis=1)
df_lung_218['linearized'] = df_lung_218.apply(lambda row: linearize(row, binary_features218, numerical_features218), axis=1)
df_lung_261['linearized'] = df_lung_261.apply(lambda row: linearize(row, binary_features261, numerical_features261), axis=1)

In [ ]:
prefix = "Patient profile:"
suffix = " Generate a concise medical summary."

def create_prompt(linearized_data):
    return f"{prefix}{linearized_data}{suffix}"

In [ ]:
def sanity_check(prompt_output, feature_name, expected_value):
    qa_prompt_categorical = f"{prompt_output}\nWhat is the value of {feature_name}?"
    qa_prompt_binary = f"{prompt_output}\nIs {feature_name} present in the above paragraph? (a) yes (b) no."
    qa_prompt_numerical =

In [11]:
import pandas as pd
import os
import openai

# Load the DataFrame (assuming df_colorectal_161 is already loaded and prepared)
data = df_colorectal_161['linearized'].tolist()

# Setup OpenAI client
client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Prepare to collect responses
responses = []

# Iterate through each data entry
for entry in data:
    prompt = create_prompt(entry)  # Make sure create_prompt function constructs the correct prompt

    # Use a try-except block for robust error handling
    try:
        # Generate the completion using the chat model
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            temperature=0.25
        )
        # Append the generated text to the responses list
        responses.append(response['choices'][0]['message']['content'])
    except Exception as e:
        print(f"Error processing data: {e}")
        # Optionally, append a placeholder or an error message
        responses.append("Error generating response")

# Add the responses as a new column to your DataFrame
df_colorectal_161['summary'] = responses

# Save the DataFrame with the summaries to a new CSV file
df_colorectal_161.to_csv('/home/alexandra/Downloads/halal/gane.csv', index=False)

print("All summaries have been generated and saved.")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}